<a href="https://colab.research.google.com/github/kusawokoshitemori/MachineLearningPractice/blob/main/MachineLearningPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 畳み込み層64,128,256 学習率0.001, アーリーストッピング変更 -> 精度99.41% (もしかしたら2層にした方が過学習抑えられていいかも)

In [ ]:
pip install keras-tuner --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
# ハーパーパラメーターの選定に使用
import keras_tuner
import keras

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

# MNISTデータを取得
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# # 訓練データを10000枚に制限
# train_images = train_images[:10000]
# train_labels = train_labels[:10000]


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# 画像データを0〜1に正規化 MLP使う場合
# train_images = train_images / 255.0
# test_images = test_images / 255.0

# 画像データを0〜1に正規化 CNN使用する場合(白黒画像だから1)
train_images = train_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0


In [ ]:
# from tensorflow.keras.utils import to_categorical

# train_labels = to_categorical(train_labels, num_classes=10)
# test_labels = to_categorical(test_labels, num_classes=10)


In [ ]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.optimizers import Adam

# モデルの関数化
def build_model(hp):
    model = keras.Sequential()

    # # ハイパーパラメーターチューニングを使用するときの畳み込み層、プーリング層　32,32,256
    # # 1つ目の畳み込み層
    # model.add(layers.Conv2D(
    #     filters=hp.Int("filters_1", min_value=32, max_value=128, step=32),
    #     kernel_size=3, activation="relu", input_shape=(28, 28, 1)
    # ))
    # model.add(layers.MaxPooling2D(pool_size=2))

    # # 2つ目の畳み込み層
    # model.add(layers.Conv2D(
    #     filters=hp.Int("filters_2", min_value=64, max_value=256, step=64),
    #     kernel_size=3, activation="relu"
    # ))
    # model.add(layers.MaxPooling2D(pool_size=2))

    # # 3つ目の畳み込み層
    # model.add(layers.Conv2D(
    #     filters=hp.Int("filters_3", min_value=128, max_value=512, step=128),
    #     kernel_size=3, activation="relu"
    # ))


    # 1つ目の畳み込み層 フィルター数変更中
    model.add(layers.Conv2D(
        filters=64,
        kernel_size=3, activation="relu", input_shape=(28, 28, 1)
    ))
    model.add(layers.MaxPooling2D(pool_size=2))

    # 2つ目の畳み込み層
    model.add(layers.Conv2D(
        filters=128,
        kernel_size=3, activation="relu"
    ))
    model.add(layers.MaxPooling2D(pool_size=2))

    # 3つ目の畳み込み層
    model.add(layers.Conv2D(
        filters=256,
        kernel_size=3, activation="relu"
    ))



    # Flattenして全結合層へ
    model.add(layers.Flatten())

    # # 隠れ層（ユニット数はハイパーパラメータ）
    # model.add(layers.Dense(
    #     units=hp.Int("units",min_value=32,max_value=64,step=32),
    #     activation="relu",
    #     kernel_regularizer=regularizers.l2(hp.Float('l2_reg', min_value=0, max_value=0.001, step=0.0001)) 0
    # ))
    # 隠れ層（ユニット数はハイパーパラメータ）
    model.add(layers.Dense(
        units=64,
        activation="relu",
        kernel_regularizer=regularizers.l2(0)
    ))

    # Dropout（過学習防止）
    model.add(layers.Dropout(0.3))

    # 出力層
    model.add(layers.Dense(10, activation="softmax"))

    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


In [ ]:
from tensorflow.keras.utils import to_categorical

# ラベルを one-hot エンコーディングに変換
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)


In [ ]:
from kerastuner.tuners import RandomSearch
import shutil

# 以前の結果を削除
shutil.rmtree("my_tuner", ignore_errors=True)

# 新しいチューナーを作成
tuner = RandomSearch(
    build_model,
    objective="val_loss",
    max_trials=15,  # 15回の試行
    executions_per_trial=1,
    directory="my_tuner",
    project_name="mnist_cnn"
)

# チューニングを実行
tuner.search(train_images, train_labels, epochs=3, validation_data=(test_images, test_labels))

# 最適なハイパーパラメータを取得
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# # 最適な L2 正則化の値を表示
# print(f"最適な L2 正則化の値: {best_hps.get('l2_reg')}")

# 最適なモデルを取得
best_model = tuner.get_best_models()[0]



Trial 25 Complete [00h 00m 53s]
val_loss: 0.10991724332173665

Best val_loss So Far: 0.044475216418504715
Total elapsed time: 00h 20m 36s

Search: Running Trial #26

Value             |Best Value So Far |Hyperparameter
0.001             |0                 |l2_reg
0.32              |0.27              |dropout_rate

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.8393 - loss: 0.5503 - val_accuracy: 0.9868 - val_loss: 0.0846


KeyboardInterrupt: 

In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
# モデルの構造を表示
best_model.summary()

# テストデータで評価
test_loss, test_acc = best_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.4f}")


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping



# EarlyStoppingの設定
# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     patience=10,
#     restore_best_weights=True
# )

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=20
    min_delta=0.001,
    restore_best_weights=True
)

# すでにコンパイル済みなら、再コンパイルは不要
# best_model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# best_model の再学習
hist = best_model.fit(train_images, train_labels,
                      epochs=200,
                      batch_size=256,
                      validation_data=(test_images, test_labels),
                      callbacks=[early_stopping])

# 学習結果の可視化
plt.plot(hist.history['accuracy'], label='Training Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# 訓練データ
train_loss, train_acc = best_model.evaluate(train_images, train_labels)
print("Train accuracy:", train_acc)



In [ ]:
# テストデータ
test_loss, test_acc = best_model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)
